## Collect a Company's Patent Porfolio

This notebook will attempt to amass, for any given company, their entire worldwide patent porfolio.\*

### U.S. Portfolio

Collecting their U.S. portfolio will proceed in a few steps:

1. Collect all applications ever assigned to the company

    a. Those applications naming the company as an applicant
    b. Those assigned to the company via an assignment
    
2. Prune all applications assigned out of the company, as recorded in USPTO assignment records

\*NOTE: I say "attempt" because the notebook relies on the USPTO system, and the International Patent Document (INPADOC) service. Not all foreign jurisdictions participate in INPADOC, and not all INPADOC member states are as good at contributing to the system as we would like. So coverage is good, but not perfect.

First, we do our standard imports, and grab the USApplication and Assignment objects from patent_client

In [46]:
import pandas as pd
from patent_client import USApplication, Assignment

company_name = 'Luminopia'

### Step 1.a: Collect all applications naming the company as applicant

In [47]:
applicant_apps = USApplication.objects.filter(first_named_applicant=company_name).values_list('appl_id', flat=True).to_list()

### Step 1.b: Collect all applications assigned to the company

In [48]:
# The assigned apps is either a single value, or a list of values if more than one property was assigned
assigned_apps = Assignment.objects.filter(assignee=company_name).explode('properties').values_list('appl_id', flat=True).to_list()

And now we combine the two

In [49]:
all_apps = set(list(applicant_apps + assigned_apps))

print(f'Total U.S. Applications Ever Owned by {company_name}: {len(all_apps)}')

Total U.S. Applications Ever Owned by Luminopia: 4


### Step 2: Collect all applications ever assigned out of the company

In [50]:
assigned_out_apps = set(Assignment.objects.filter(assignor=company_name).to_pandas()
                   .query('conveyance_text == "ASSIGNMENT OF ASSIGNORS INTEREST"')
                   .properties.explode().apply(lambda x: x.appl_id).to_list())
print(f'U.S. Applications assigned out of {company_name}: {len(assigned_out_apps)}')

U.S. Applications assigned out of Luminopia: 1


### Step 3: Subtract one set from the other

In [51]:
owned_apps = list(set(all_apps) - set(assigned_out_apps))
len(owned_apps)


4

In [52]:
owned_apps

['16420557', 'PCT/US17/27629', None, '62323284']

### Step 4: Generate status report

In [53]:
import pandas as pd
from patent_client.parser import parse

us_df = USApplication.objects.filter(appl_id=owned_apps).values(
    'appl_id', 
    'app_filing_date', 
    'patent_number',
    'patent_issue_date',
    'patent_title',
    'inventors.0.name',
    'applicants.0.name',
    'app_status',
    'app_status_date',
    'expiration.initial_term',
    'expiration.pta_or_pte',
    'expiration.extended_term',
    'expiration.terminal_disclaimer_filed',
).to_pandas()

us_df

,appl_id,app_filing_date,patent_number,patent_issue_date,patent_title,inventors.0.name,applicants.0.name,app_status,app_status_date,expiration.initial_term,expiration.pta_or_pte,expiration.extended_term,expiration.terminal_disclaimer_filed
0,16420557,2019-05-23,None,None,Visual disorder treatment,Travers; Dean,"Luminopia, Inc.;",Final Rejection Mailed,2022-07-12,None,None,None,None
1,PCT/US17/27629,2017-04-14,None,None,METHODS AND HEAD-MOUNTED APPARATUS FOR VISUAL ...,WENDLAND; ALEXANDER,"LUMINOPIA, INC.;",RO PROCESSING COMPLETED-PLACED IN STORAGE,2017-04-20,None,None,None,None
2,62323284,2016-04-15,None,None,Treatment of Visual Disorders and Conditions w...,Travers; Dean,"Luminopia, Inc.;",Provisional Application Expired,2017-04-16,None,None,None,None


### Foreign Patent Portfolio

Now we will fetch the company's wordwide portfolio, as it appears in the Inpadoc system maintained by the EPO

In [54]:
from patent_client import Inpadoc

foreign = Inpadoc.objects.filter(applicant=company_name)

And now, a status report:

In [55]:
foreign_df = foreign.values(
    'country', 
    appl_id='biblio.application_reference_epodoc.number', 
    app_filing_date='biblio.application_reference_epodoc.date', 
    pub_number='biblio.publication_reference_epodoc.number', 
    pub_date='biblio.publication_reference_epodoc.date', 
    title='biblio.title', 
    applicant='biblio.applicants_epodoc.0', 
    first_named_inventor='biblio.inventors_epodoc.0').to_pandas()
foreign_df = foreign_df[foreign_df['country'] != 'US']
foreign_df

,country,appl_id,app_filing_date,pub_number,pub_date,title,applicant,first_named_inventor
0,WO,WO2017US27629,2017-04-14,WO2017181010,2017-10-19,METHODS AND HEAD-MOUNTED APPARATUS FOR VISUAL ...,LUMINOPIA INC [US],TRAVERS DEAN [US]


And now we will take our results, and write to an excel file

In [45]:
writer = pd.ExcelWriter(f'/Users/parkerhancock/{company_name}_portfolio.xlsx')
us_df.to_excel(writer, sheet_name='US Portfolio')
foreign_df.to_excel(writer, sheet_name='Foreign Portfolio')
writer.save()

/var/folders/bt/s4hv2ltx71nf43jxmddn4s5h0000gn/T/ipykernel_11834/622620193.py:4: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
